<a href="https://colab.research.google.com/github/jcandane/CellList/blob/main/CellList_modmod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Given R_ix generate cell-list 
### cell-list (box-box neighbors and which atoms, i, belong with which box.)

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import time

np.set_printoptions(precision=4, linewidth=200, threshold=2000, suppress=True)

In [26]:
!git clone https://github.com/jcandane/pykry
from pykry.fcc import FCClattice

fatal: destination path 'pykry' already exists and is not an empty directory.


#### Cell List Def.

In [27]:
def Cell_List(R_ix, cutoff, boxboxcuttoff=1.5):

    R_ix_int = ( R_ix/cutoff ).astype(int)
    lims     = 1 + np.amax( R_ix_int, axis=0 ) - np.amin( R_ix_int, axis=0 )
    boxes    = np.arange(0, np.prod(lims), 1, dtype=int).reshape((lims[0],lims[1],lims[2]))

    box = boxes[R_ix_int[:,0], R_ix_int[:,1], R_ix_int[:,2]] ## get box for each atom
    ind = np.argsort(box) ## find indices beloning to a box
    sor = np.append([0], np.where( np.diff( box[ind] ) == 1 )[0] ) ### find where boxes end, i.e. box[ind] = np.sort( box )
    sor = np.append(sor, box[-1] ) ## !!

    x_  = np.arange(0, lims[0], 1, dtype=int)
    y_  = np.arange(0, lims[1], 1, dtype=int)
    z_  = np.arange(0, lims[2], 1, dtype=int)
    xyz = np.array(np.meshgrid(x_, y_, z_))

    R_Bx  = (xyz.swapaxes(0,3)).reshape((xyz.size//xyz.shape[0] , xyz.shape[0]), order="F")
    R_BCx = np.einsum("Bx, C -> BCx", R_Bx, np.ones(len(R_Bx), dtype=int)) - np.einsum("Cx, B -> BCx", R_Bx, np.ones(len(R_Bx), dtype=int))
    Boxdistances = np.einsum("BCx -> BC", R_BCx**2)**0.5
    II, JJ = np.where( np.logical_and(np.triu( Boxdistances ) > 0, np.triu( Boxdistances ) < boxboxcuttoff) )
    ## IK = IJ[ np.where(IJ[:,0] <= IJ[:,1])[0], : ]

    return np.asarray([II, JJ]).T, ind, sor

## Get the System: R_ix, cutoff

In [28]:
N_atoms = 200000
cutoff  = 10

size     = np.array([100., 100., 200.])
R_ix     = size[None, :]*(np.random.random((N_atoms,3)) - 1/2) ## 100 is the size
print("Number-Density = " + str(N_atoms / size**3) ) ## number/vol = number-density, diamond = 0.1762 1/Å^3 see https://en.wikipedia.org/wiki/Number_density

Number-Density = [0.2   0.2   0.025]


In [29]:
ZZ, R_ix = FCClattice(5.26, 15, 15, 15, element="Ar")

In [63]:
R_ix ## in Å
len(R_ix)

13500

In [38]:
cube_size = 16 ## Å
R_ix_int  = ( R_ix/cube_size ).astype(int)
lims      = 1 + np.amax( R_ix_int, axis=0 ) - np.amin( R_ix_int, axis=0 )
boxes     = np.arange(0, np.prod(lims), 1, dtype=int).reshape((lims[0],lims[1],lims[2]))

print(lims)
print(boxes.shape)

[5 5 5]
(5, 5, 5)


In [78]:
box = boxes[R_ix_int[:,0], R_ix_int[:,1], R_ix_int[:,2]] ## get box for each atom
ind = np.argsort(box) ## find indices beloning to a box
sor = np.append([0], np.where( np.diff( box[ind] ) >= 1 )[0] ) ### find where boxes end, i.e. box[ind] = np.sort( box ) !!! >=
sor = np.append(sor, len(box) ) ## !!

print(box.shape)
print(ind.shape)
print(boxes.size)
print(sor.shape)

ssss = 0
for i in range(boxes.size):
    ssss += len( ind[ sor[(i)]:sor[(i)+1] ] )

ssss

(13500,)
(13500,)
125
(126,)


13500

In [32]:
max  = np.asarray([ np.amax( R_ix_int[:,0] ), np.amax( R_ix_int[:,1] ), np.amax( R_ix_int[:,2] )])
min  = np.asarray([ np.amin( R_ix_int[:,0] ), np.amin( R_ix_int[:,1] ), np.amin( R_ix_int[:,2] )])
limz = 1 + max - min

boxes    = np.arange(0, np.prod(lims), 1, dtype=int).reshape((limz[0],limz[1],limz[2]))

In [33]:
def givenbox_getatoms(IJ, cell_list_info):
    indexes, sorted = cell_list_info
    i = indexes[ sorted[IJ[0]]:sorted[IJ[0]+1] ]
    j = indexes[ sorted[IJ[1]]:sorted[IJ[1]+1] ]
    return i,j

### Lets Calculate the Cell List

In [34]:
IJ, indexes, sorted = Cell_List(R_ix, cutoff) ### IJ is NOT double counting, e.g. doesn't have 456, 34 only 34, 456
cell_list_info = [indexes, sorted]

I = 9
J = 11
boxI_i = indexes[ sorted[I]:sorted[I+1] ] ## atom indices belonging to box I
boxJ_j = indexes[ sorted[J]:sorted[J+1] ] ## atom indices belonging to box J

In [35]:
dist = 0.
for i in range(len(IJ)):
    I, J  = IJ[i]
    I_i   = indexes[ sorted[I]:sorted[I+1] ] ## atom indices belonging to box I
    J_j   = indexes[ sorted[J]:sorted[J+1] ] ## atom indices belonging to box J

    R_ijx = R_ix[ I_i, None, : ] - R_ix[ None, J_j, : ]
    dist += np.sum( R_ijx ) + np.sum( -R_ijx.swapaxes(0,1) )

    if len(I_i) == 0:
      print("zero atoms in box " + str(i))

print(dist)

0.0


In [36]:
!git clone https://github.com/jcandane/pyprototyp
from pyprototyp.pyprototyp import pyprototyp
pyprototyp("CellList", packages=[], useful=[], defs=[Cell_List], classes=[])

fatal: destination path 'pyprototyp' already exists and is not an empty directory.
